In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import csv
import os
import itertools

In [2]:
from scripts.utility import *
from scripts.features import *
from scripts.decomposer import hierarchical_decomposition

# Features
It is a good idea to save our features in text files, so that we only have to calculate them once. Later on when we employ clustering, we can just read the values off of these files instead of iterating over the graphs again, which is time consuming.

## 1. Geometrical

### 1.1 Number of nodes and edges

In [ ]:
# Calculate the number of nodes and edges for each network and save them to files
with open('features/n_nodes.txt', 'w') as node_file, open('features/n_edges.txt', 'w') as edge_file:
    for network_id, G in graph_generator():
        node_file.write(network_id + '\t' + str(nx.number_of_nodes(G)) + '\n') # nx.number_of_nodes() function
        edge_file.write(network_id + '\t' + str(nx.number_of_edges(G)) + '\n') # use tab character \t, not two spaces
        
###NOTE: Have to pull features from github before running script
##IDEA: Can calculate every feature on the same loop

### 1.2 Average node degree

In [ ]:
# Calculate the average node degree, i.e. the average number of edges that are connected to a node,
# for each network and write them to a file
with open('features/average_node_degree.txt', 'w') as file:
    for network_id, G in graph_generator(): # check this access to graph_generator
        average_node_degree = np.mean(list(G.degree().values()))
        file.write(network_id + '\t' + str(average_node_degree) + '\n')

### 1.3 Vein density

Vein density **$\sigma$** is calculated by summing the total length of all veins and dividing by total leaf area.


In [ ]:
save_feature(vein_density, skip_existing=True, clean=False)

### 1.4 Areole area

Areole area **$A$** is obtained calculating basic cycle areas using the standard formula for the area of a general polygon.

In [3]:
save_feature(areole_area, skip_existing=True, clean=False)

Removing disconnected parts
Saving areole_area for BronxA_001...
Removing disconnected parts
Saving areole_area for BronxA_002...
Removing disconnected parts
Saving areole_area for BronxA_003...
Removing disconnected parts
Saving areole_area for BronxA_004...
Removing disconnected parts
Saving areole_area for BronxA_005...
Removing disconnected parts
Saving areole_area for BronxA_006...
Removing disconnected parts
Saving areole_area for BronxA_007...
Removing disconnected parts
Saving areole_area for BronxA_008...
Removing disconnected parts
Saving areole_area for BronxA_009...
Removing disconnected parts
Saving areole_area for BronxA_010...
Removing disconnected parts
Saving areole_area for BronxA_014_a...
Removing disconnected parts
Saving areole_area for BronxA_014_b...
Removing disconnected parts
Saving areole_area for BronxA_014_c...
Removing disconnected parts
Saving areole_area for BronxA_014_d...
Removing disconnected parts
Saving areole_area for BronxA_014_e...
Removing discon

### 1.5 Areole density 

Areole density **$\rho_A$** is the total number of areoles divided by leaf area.

In [4]:
save_feature(areole_density, skip_existing=True, clean=False)

Removing disconnected parts
Saving areole_density for BronxA_001...
Removing disconnected parts
Saving areole_density for BronxA_002...
Removing disconnected parts
Saving areole_density for BronxA_003...
Removing disconnected parts
Saving areole_density for BronxA_004...
Removing disconnected parts
Saving areole_density for BronxA_005...
Removing disconnected parts
Saving areole_density for BronxA_006...
Removing disconnected parts
Saving areole_density for BronxA_007...
Removing disconnected parts
Saving areole_density for BronxA_008...
Removing disconnected parts
Saving areole_density for BronxA_009...
Removing disconnected parts
Saving areole_density for BronxA_010...
Removing disconnected parts
Saving areole_density for BronxA_014_a...
Removing disconnected parts
Saving areole_density for BronxA_014_b...
Removing disconnected parts
Saving areole_density for BronxA_014_c...
Removing disconnected parts
Saving areole_density for BronxA_014_d...
Removing disconnected parts
Saving areol

### 1.6 Weighted vein thickness

Weighted vein thickness **$d$** is calculated as the total sum of the product radius*length of each
    individual vein segment divided by total vein length

In [5]:
save_feature(weighted_vein_thickness, skip_existing=True, clean=False)

Removing disconnected parts
Saving weighted_vein_thickness for BronxA_001...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_002...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_003...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_004...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_005...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_006...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_007...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_008...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_009...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_010...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_014_a...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_014_b...
Removing disconnected parts
Saving weighted_vein_thickness for BronxA_01

## 2. Topological

### 2.1 Nesting number

In [ ]:
save_feature(nesting_numbers, skip_existing=True, clean=True)

Removing disconnected parts
Removing intersecting edges.
((134, 42679), (42674, 42687))
((4290, 39177), (26052, 4293))
((4293, 26052), (39177, 4290))
((7904, 31530), (31245, 31540))
((24487, 26256), (26253, 26082))
((25091, 37417), (37419, 37432))
((25373, 25417), (31880, 25421))
((25421, 31880), (25373, 25417))
((25989, 26048), (34839, 26045))
((26045, 34839), (25989, 26048))
((26082, 26253), (26256, 24487))
((27625, 36060), (42121, 27637))
((27637, 42121), (36060, 27625))
((31245, 31540), (31530, 7904))
((37419, 37432), (25091, 37417))
((42674, 42687), (134, 42679))
Pruning.
Applying workaround to remove spurious collinear edges.
Removing offending edges.
Pruning again.
Connected components: 5 

Saving nesting_numbers for BronxA_001...
Detecting minimal cycles.
Number of cycles including boundary: 8439.


/home/hackerman/Desktop/Göttingen Winter Semester 2016-2/Machine learning/Leaf architecture Project/Leaf-architecture /scripts/cycle_basis.py:71: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if coords == None:


Detected fundamental cycles in 12.434550285339355s
Constructing dual.
Pruning dual.
Dual connected components: 1.
Detecting outermost loop and rewiring.
Performing hierarchical decomposition.


In [ ]:
%%capture 
print('alan')

# Data Analysis

In [ ]:
generator = graph_generator()

In [ ]:
_, G = next(generator)

In [ ]:
H = clean_graph(G)

In [ ]:
pos = nx.get_node_attributes(G, 'pos')
nx.draw(G, pos=pos, node_size=0.01)

#basis = nx.cycle_basis(G)
#subgraph_nodes = list(itertools.chain(*basis[:700])) # TODO: how to generate connected subgraphs?
#H = clean_graph(G.subgraph(subgraph_nodes))
#pos = nx.get_node_attributes(H,'pos')
#nx.draw(H, pos=pos, node_size=0.0001)

In [ ]:
nesting_number, nesting_number_no_ext = nesting_numbers(H)
print(nesting_number)
print(nesting_number_no_ext)

In [ ]:
_, G = get_graph_from_data()